In [ ]:
import os
import toml
import json
import requests
from requests.exceptions import SSLError
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from dotenv import load_dotenv

# Suppress unverified HTTPS warnings, but keep others
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Load environment variables from .env file
load_dotenv()


class APICClient:
    def __init__(self, config_path):
        self.config = self.load_config(config_path)
        self.username, self.password = self.load_api_credentials()

    @staticmethod
    def load_config(config_path):
        return toml.load(config_path)

    @staticmethod
    def load_api_credentials():
        username = os.getenv("API_USERNAME")
        password = os.getenv("API_PASSWORD")
        return username, password

    def login(self, environment):
        api_base_url = self.config["api"][environment]
        login_endpoint = self.config["endpoint"]["login"]
        login_url = f"{api_base_url}{login_endpoint}"

        session = requests.Session()

        try:
            response = session.post(
                login_url,
                json={
                    "aaaUser": {
                        "attributes": {"name": self.username, "pwd": self.password}
                    }
                },
                verify=False,
            )
        except (Exception, SSLError) as e:
            print(f"Login error for environment {environment}: {e}")
            return None, f"Login error: {e}"

        if response.status_code != 200:
            print(
                f"Failed to authenticate with the APIC for environment {environment}. Status code: {response.status_code}"
            )
            return (
                None,
                f"Failed to authenticate with the APIC. Status code: {response.status_code}",
            )

        login_data = response.json()
        token = login_data["imdata"][0]["aaaLogin"]["attributes"]["token"]
        session.cookies.set("APIC-cookie", token)

        print(f"Successfully logged in to {environment}")
        return session, {"APIC-cookie": token}

    def fetch_data(self, session, cookie, environment, endpoint_key, **url_params):
        if session is None:
            return None

        api_base_url = self.config["api"][environment]
        api_endpoint = self.config["endpoint"][endpoint_key]
        url = f"{api_base_url}{api_endpoint.format(**url_params)}"

        try:
            response = session.get(url, cookies=cookie, verify=False)
            response.raise_for_status()
            response_json = response.json()
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}")
            return None
        except Exception as err:
            print(f"Other error occurred: {err}")
            return None

        return response_json

    def get_active_leaf_dns(self, session, cookie, environment, pod):
        # Fetch active nodes from the node endpoint
        nodes_json = self.fetch_data(session, cookie, environment, "node", pod=pod)
        if nodes_json is None:
            print(f"No data returned for environment: {environment}, pod: {pod}")
            return [], None

        acceptable_roles = {"leaf", "spine", "tier-2-leaf", "remote-leaf"}
        leaf_nodes = [
            {
                "id": node["fabricNode"]["attributes"]["id"],
                "role": node["fabricNode"]["attributes"]["role"],
            }
            for node in nodes_json.get("imdata", [])
            if node["fabricNode"]["attributes"]["role"] in acceptable_roles
        ]
        return leaf_nodes, nodes_json


# Configuration for the script
config_path = "api.toml"  # Path to your actual config path
environments = ["lab", "prod"]  # Environments to fetch data from
pods = ["1", "2"]  # Example pod parameters (adjust based on your needs)
output_json_path = (
    "/mnt/edl/raw/iit_apic_raw/unprocessed/APIC.json"  # Path to the output JSON file
)

all_responses = {"environments": {}}

# Instantiate the client and perform operations
client = APICClient(config_path)

for environment in environments:
    session, cookie = client.login(environment)
    if session:
        all_responses["environments"][environment] = {"pods": {}}
        for pod in pods:
            print(f"Fetching data for environment: {environment}, pod: {pod}")
            all_responses["environments"][environment]["pods"][pod] = {
                "nodes": None,
                "leaf_nodes_data": [],
            }

            # Get all active leaf DNs and node data
            leaf_nodes, nodes_json = client.get_active_leaf_dns(
                session, cookie, environment, pod
            )
            all_responses["environments"][environment]["pods"][pod][
                "nodes"
            ] = nodes_json

            if not leaf_nodes:
                print(f"No leaf nodes found for environment: {environment}, pod: {pod}")
                continue

            # Iterate through each leaf node and fetch data
            for leaf_node in leaf_nodes:
                leaf_dn = leaf_node["id"]
                node_role = leaf_node["role"]
                leaf_data = {"leaf_dn": leaf_dn, "role": node_role}
                url_params = {"pod": pod, "leaf_dn": leaf_dn}

                # Fetch line card data only for nodes with role 'spine'
                if node_role == "spine":
                    linecard_data = client.fetch_data(
                        session, cookie, environment, "linecard", **url_params
                    )
                    if linecard_data:
                        leaf_data["linecard"] = linecard_data

                # If leaf_data contains only 'leaf_dn' and 'role', skip adding it to the result
                if len(leaf_data) > 2:
                    all_responses["environments"][environment]["pods"][pod][
                        "leaf_nodes_data"
                    ].append(leaf_data)
    else:
        print(f"Failed to log in to {environment}")

# Convert the all_responses dictionary to a JSON string
all_responses_json = json.dumps(all_responses, indent=4)

# Write the JSON string to the specified path using dbutils
dbutils.fs.put(output_json_path, all_responses_json, overwrite=True)

print(f"All data have been saved to {output_json_path}.")